# Summary

In [21]:
from abstractor.train import get_training_batch as get_abstractor_training_batch
from abstractor.utils import AbstractorModel, AbstractorModelRNN
from abstractor.utils import obtain_initial_hidden_states
from bert.utils import obtain_sentence_embeddings
from bert.utils import obtain_word_embeddings
from data.utils import load_training_dictionaries
from extractor.train import get_training_batch as get_extractor_training_batch
from extractor.utils import ExtractorModel
from pytorch_transformers import BertModel
from pytorch_transformers import BertTokenizer
from rl_connection.utils import RLModel
from rl_connection.train import get_training_batch as get_rl_training_batch
from rouge import Rouge

import numpy as np
import torch

In [22]:
def print_results(results):
    for x in results:
        print(f'{x}\n')

## Load data

In [23]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [24]:
data = load_training_dictionaries()

## Extractor

In [49]:
# Load extractor model:
extractor_model = ExtractorModel(bert_tokenizer, bert_model)
extractor_model_path = "results/models/extractor.pt"
extractor_model.load_state_dict(torch.load(extractor_model_path))

<All keys matched successfully>

In [50]:
documents, extraction_labels = get_extractor_training_batch(data, batch_size=4)

sentence_embeddings, sentence_mask = obtain_sentence_embeddings(
    extractor_model.bert_model, 
    extractor_model.bert_tokenizer, 
    documents
)

# Predict probability of extraction per sentence
extraction_probabilities, extraction_mask = extractor_model(sentence_embeddings, sentence_mask, extraction_labels)

In [51]:
for batch_idx, ext_probs in enumerate(extraction_probabilities):
    ext_sent_indicies = torch.argmax(ext_probs, dim=1)[:extraction_mask[batch_idx].sum()]
    predicted_ext_sents = np.array(documents[batch_idx])[ext_sent_indicies]
    target_ext_sents = np.array(documents[batch_idx])[extraction_labels[batch_idx][:len(documents[batch_idx])].numpy().astype(bool)]
    
    print("############### PREDICTIONS: ###############")
    print_results(predicted_ext_sents)
    print("############### TARGETS: ###############")
    print_results(target_ext_sents)
    print('--------------------------------------------------------')
    print('--------------------------------------------------------')
    print('--------------------------------------------------------\n')
    

############### PREDICTIONS: ###############
but when rory mcilroy pulled his second shot on the eighth hole of the wgc cadillac championship into a lake friday , he might as well have been channeling the much loved adam sandler character .

before continuing his round with a dropped ball , the four - time major winner launched the 3 - iron used to play the offending shot into the water as well .

############### TARGETS: ###############
but when rory mcilroy pulled his second shot on the eighth hole of the wgc cadillac championship into a lake friday , he might as well have been channeling the much loved adam sandler character .

before continuing his round with a dropped ball , the four - time major winner launched the 3 - iron used to play the offending shot into the water as well .

--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------

############### PREDICTIONS: #

## Abstractor

In [ ]:
# Load data:
abstractor_model = AbstractorModelRNN(bert_tokenizer, bert_model)
abstractor_model_path = "results/models/abstractor.pt"
abstractor_model.load_state_dict(torch.load(abstractor_model_path))

In [ ]:
source_documents, target_summaries = get_abstractor_training_batch(data, 1)

# Obtain embeddings
source_document_embeddings, source_mask, source_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, source_documents, static_embeddings=False
)
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, target_summaries, static_embeddings=True
)

print(source_documents)
print()
print(target_summaries)

In [ ]:
# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=0
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")
    
print("------------")


# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=1
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")

## Reinforcement Learning

In [ ]:
rl_model = RLModel(extractor_model, abstractor_model)
rl_model.load_state_dict(torch.load("results/models/rl.pt"))

In [ ]:
source_documents, target_summaries = get_rl_training_batch(data, batch_size=1)

# Obtain embeddings
source_sentence_embeddings, source_mask = obtain_sentence_embeddings(
    rl_model.extractor_model.bert_model,
    rl_model.extractor_model.bert_tokenizer,
    source_documents
)
stop_action_index = source_sentence_embeddings.shape[1]
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    rl_model.abstractor_model.bert_model,
    rl_model.abstractor_model.bert_tokenizer,
    target_summaries,
    static_embeddings=True
)

# Run trajectory
actions, log_probs, entropys, values, n_ext_sents = rl_model.sample_actions(
    source_sentence_embeddings,
    source_mask
)

# Obtain abstracted sentence from abstractor
predicted_tokens, word_probabilities = rl_model.create_abstracted_sentences(
    actions,
    source_documents,
    n_ext_sents=n_ext_sents,
    teacher_forcing_pct=0,
    target_summary_embeddings=target_summary_embeddings
)

# Look at extractions
for art_idx, doc_sentences in enumerate(actions):
    for sent_idx in doc_sentences[:-1]:
        print(source_documents[art_idx][sent_idx])
        print()
    print("\n\n-------\n\n")
    
# Look at abstractions
for predicted_abstraction in predicted_tokens:
    solution = list()
    for token in predicted_abstraction:
        solution.append(rl_model.abstractor_model.bert_tokenizer.ids_to_tokens[int(token)])
    print(" ".join(solution))
    print("\n\n")
    
print("-------\n\n")
target_summaries

## Scratch

In [15]:
action_probs = torch.rand(4, 1, 16)
extraction_mask = torch.ones(action_probs.shape)
action_indicies = [torch.tensor([[12, 0, 1, 2]]), torch.tensor([[11, 6, 9, 4]])]
action_indicies = torch.cat(action_indicies).T

In [20]:
extraction_mask[[[0], [1], [2], [3]], 0, action_indicies]

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [4], [4, 2]

In [10]:
torch.empty(init_sent_embedding.shape)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
print(a.shape)
print(b.shape)

In [ ]:
c = a + b
print(c)
c.shape

In [ ]:
from collections import defaultdict

label_indicies_dict = defaultdict(list)
for row, pos in zip(b, c):
    label_indicies_dict[row.item()].append(pos.item())

In [ ]:
label_indicies_dict

In [ ]:
a[0][[0, 0, 0]]

In [ ]:
a = torch.rand(7, 3)
b = torch.rand(4, 3)

In [ ]:
print(a)
print()
print(b)

In [ ]:
-np.log(0.999)

In [ ]:
a[0] + b